# Input

In [ ]:
target_column = "RecommendHiring"
%load_ext autoreload
%autoreload 2
VIDEOS_FOLDER = "./data/raw/videos"
import importlib.util
from pathlib import Path
import os
analyzer_paths = Path("./src/utils").resolve()

# Constants

In [ ]:
import os
import numpy as np
from sklearn.metrics import make_scorer

SCRIPT_DIR = os.path.join(os.getcwd(),)

SAVED_MODELS_PATH="./saved models"

DROPPED_LEXICAL_COLUMNS = [
    "Swear",
    "Numbers",
    "Inhibition",
    "Preceptual",
    "Anxiety",
    "Anger",
    "Sadness",
    "Work",
    "Articles",
    "Verbs",
    "Adverbs",
    "Prepositions",
    "Conjunctions",
    "Negations",
]

facial_features = [
    "average_inner_brow_height",
    "average_outer_brow_height",
    "eye_open",
    "inner_lip_height",
    "lip_corner_distance",
    "outer_lip_height",
    "smile",
    "pitch",
    "roll",
    "yaw",
]

stats = ["max", "median", "min", "std"]
DROPPED_FACIAL_FEATURES = [
    f"{feature}_{stat}" for feature in facial_features for stat in stats
]


DROPPED_PROSODIC_COLUMNS = []

ALREADY_NORMALIZED_FEATURES = [
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "eye_open_mean",
    "inner_lip_height_mean",
    "inner_lip_height_mean",
    "lip_corner_distance_mean",
    "average_outer_brow_height_std",
    "average_inner_brow_height_std",
    "eye_open_std",
    "outer_lip_height_std",
    "inner_lip_height_std",
    "lip_corner_distance_std",
    "average_outer_brow_height_min",
    "average_inner_brow_height_min",
    "eye_open_min",
    "outer_lip_height_min",
    "inner_lip_height_min",
    "lip_corner_distance_min",
    "average_outer_brow_height_max",
    "average_inner_brow_height_max",
    "eye_open_max",
    "outer_lip_height_max",
    "inner_lip_height_max",
    "lip_corner_distance_max",
    "average_outer_brow_height_median",
    "average_inner_brow_height_median",
    "eye_open_median",
    "outer_lip_height_median",
    "inner_lip_height_median",
    "lip_corner_distance_median",
]  # these are already in [0, 1]

MUST_KEEP_FEATURES = [
    "pause_duration_avg",
    "average_outer_brow_height_mean",
    "average_inner_brow_height_mean",
    "outer_lip_height_mean",
    "Duration/Filler Words",
]


GROUPS_COLUMN = "cleaned_ids"
INDEX_COLUMN = "participant_id"


def pearson_corr(y_true, y_pred):
    return np.corrcoef(y_true, y_pred)[0, 1]


SCORING_METRICS = {
    "r2": "r2",
    "mae": "neg_mean_absolute_error",
    "pearson": make_scorer(pearson_corr),  # Pearson Correlation Coefficient
}


MUST_KEEP_FEATURES = [
    # "pause_duration_avg",
    # "average_outer_brow_height_mean",
    # "average_inner_brow_height_mean",
    # "outer_lip_height_mean",
    "Duration/Filler Words",
]

PIPELINE_PARAMS = {'feature_selection__estimator__alpha': 0.057376790661083456, 'svr__C': 0.655379988356498, 'svr__gamma': 0.02784736494309893, 'svr__epsilon': 0.2617249201838037, 'svr__kernel': 'rbf'}
HYPERPARAMETER_TUNING_ENABLED = True

# Data Preprocessing

## Import Datasets

In [ ]:
import pandas as pd
import os

features_df = pd.read_csv("../../data/external/add.csv")
features_df = features_df.set_index("participant_id")

labels_df = pd.read_csv(
    os.path.join("../../data/external/turker_scores_full_interview.csv")
)
labels_df = labels_df.set_index("Participant")
labels_df = labels_df.loc[labels_df["Worker"] == "AGGR"]

features_df.index = features_df.index.str.lower()
labels_df.index = labels_df.index.str.lower()
indexed_combined_df = features_df.join(labels_df[[target_column]], how="left")
combined_df = indexed_combined_df.reset_index(drop=True)

# Model

## Split Data

In [ ]:
X = combined_df.drop(columns=[target_column, GROUPS_COLUMN])
y = combined_df[target_column]

## Pipeline Creation

In [ ]:
import numpy as np
from sklearn.compose import ColumnTransformer
from sklearn.feature_selection import SelectFromModel
from sklearn.impute import SimpleImputer
from sklearn.linear_model import Lasso
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.svm import SVR
from sklearn.model_selection import GroupKFold, cross_val_score, cross_validate
import sys

sys.path.append("..")

import models.domain_aware_selector
groups_column = combined_df[GROUPS_COLUMN].astype(str).values


preprocessor = ColumnTransformer(
    [
        ('dropper', 'drop', DROPPED_FACIAL_FEATURES + 
                            DROPPED_LEXICAL_COLUMNS + 
                            DROPPED_PROSODIC_COLUMNS)
    ],
    remainder='passthrough'
)
unfitted_pipeline = Pipeline(
    [
        ('preprocessor', preprocessor),
        ("imputer", SimpleImputer(strategy="mean")),  # NaN imputation
        ("scaler", StandardScaler()),
        ("feature_selection", SelectFromModel(estimator=Lasso(max_iter=50000), )),
        # ('feature_selection', DomainAwareSelector(
        #     must_keep_features=MUST_KEEP_FEATURES,
        #     selector=SelectFromModel(lasso_feature_selection_model, max_features=10),
        # )),
        ("svr", SVR(kernel="rbf")),
    ]
)

## Hyperparameter Tuning

In [ ]:
import optuna
from sklearn.base import clone
from sklearn.model_selection import GroupShuffleSplit, cross_val_score
import numpy as np

def objective(trial):
    pipeline_clone = clone(unfitted_pipeline)  # Clone pipeline for thread safety
    
    params = {
        "feature_selection__estimator__alpha": trial.suggest_float(
        "feature_selection__estimator__alpha", 1e-4, 0.1, log=True  # Adjusted lower bound
        ),
        "svr__C": trial.suggest_float("svr__C", 0.01, 100, log=True),
        "svr__gamma": trial.suggest_float("svr__gamma", 1e-3, 1e1, log=True),
        "svr__epsilon": trial.suggest_float("svr__epsilon", 0.01, 0.5),
        "svr__kernel": trial.suggest_categorical("svr__kernel", ["rbf"
                                                                #  , "poly"
                                                                 ]),
    }
    
    # if params["svr__kernel"] == "poly":
    #     params["svr__degree"] = trial.suggest_int("svr__degree", 2, 3)  # Reduced from 5
    #     params["svr__coef0"] = trial.suggest_float("svr__coef0", 0.0, 0.5)  # Narrower range
        
    pipeline_clone.set_params(**params)
    
    mc_cv_tuning = GroupShuffleSplit(n_splits=20, test_size=0.2, random_state=42)
    
    scores = cross_val_score(
        pipeline_clone, X, y, cv=mc_cv_tuning, groups=groups_column, n_jobs=1
    )

    return np.mean(scores)

if HYPERPARAMETER_TUNING_ENABLED:
    study = optuna.create_study(direction="maximize")
    study.optimize(objective, n_trials=100, n_jobs=-1, timeout=4*60)
    
    print("Best hyperparameters:", study.best_params)
    print(f"Best R² score: {study.best_value:.4f}")

## Store Trained Model

In [ ]:
from sklearn import clone

if HYPERPARAMETER_TUNING_ENABLED:
    unfitted_pipeline.set_params(**study.best_params)
else:
    unfitted_pipeline.set_params(**PIPELINE_PARAMS)
fitted_pipeline = clone(unfitted_pipeline)
fitted_pipeline.fit(X, y)

## Feature Selection Results

In [ ]:
preprocessor = fitted_pipeline.named_steps['preprocessor']
feature_names = preprocessor.get_feature_names_out()    # after preprocessing

feature_selector = fitted_pipeline.named_steps['feature_selection']
selected_mask = feature_selector.get_support()

selected_features = feature_names[selected_mask]
unselected_features = feature_names[~selected_mask]

print(f"Number of Selected features ({len(selected_features)}):")
print(f"Selected features: {selected_features}")
print(f"Unselected features: {unselected_features}")

# Monte Carlo Cross Validation

In [ ]:
from sklearn.model_selection import cross_validate
from sklearn.metrics import r2_score

scoring = {
    "r2": make_scorer(r2_score),
    "pearson": make_scorer(pearson_corr)
}

results = cross_validate(
    unfitted_pipeline,
    X,
    y,
    cv=GroupShuffleSplit(n_splits=1000, test_size=0.2, random_state=42),
    groups=groups_column,
    scoring=scoring,
    n_jobs=-1,
)

r2_scores = results["test_r2"]
pearson_scores = results["test_pearson"]
# Report the average performance and variability
avg_r2_score = np.mean(r2_scores)
print(
    f"Mean R² Score: {avg_r2_score:.2f} (±{np.std(r2_scores):.2f})"
)
avg_perason_score = np.mean(pearson_scores)
print(
    f"Mean Pearson Correlation: {avg_perason_score:.2f} (±{np.std(pearson_scores):.2f})"
)

# Mean R² Score: 0.20 (±0.23)
# Mean Pearson Correlation: 0.53 (±0.16)

# Save the Model

In [ ]:
from joblib import dump, load

dump(fitted_pipeline, os.path.join(SAVED_MODELS_PATH,f'{target_column}.joblib'))